# Exercise 2.1

In [1]:
using DistMesh

In [2]:
function xy(x0::Float64, y0::Float64, L1::Float64, L2::Float64, noelms1::Int64, noelms2::Int64)
    # Computes the x-coordinates and y-coordinates in a given range, with a given grid definition.
    # Author: Clara Hollenbeck
    # x0, y0: minimum x and y value
    # L1, L", maximum x and y value respectively
    # noelms1, noelms2: no. of elements in the x and y direction respectively.

    VX= repeat(collect(LinRange(x0, x0 + L1, noelms1+1)), inner = noelms2+1)
    VY = repeat(collect(LinRange(y0 + L2, y0, noelms2+1)), noelms1+1)
    return VX, VY
end

xy (generic function with 1 method)

In [3]:
function conelmtab(noelms1::Int64, noelms2::Int64)
    novertices = noelms1 *noelms2*2
    EToV = zeros(Int64, novertices, 3)

    for i in 1: noelms1
        v1 = collect(range(1, noelms2)) .+ (noelms2+1)*(i-1)
        v2 = collect(range(noelms2 + 3, 2*noelms2+2)) .+ (noelms2+1)*(i-1)
        v3 = collect(range(noelms2 + 2, 2*noelms2 +1)) .+ (noelms2+1)*(i-1)
        v4 = collect(range(2 , noelms2+1)) .+ (noelms2+1)*(i-1)

        
        EToV[1 + noelms2*2 *(i-1) : noelms2*2 *i , :] = vcat(hcat(v1,v2,v3), hcat(v1, v4, v2))

    end
    return EToV[sortperm(EToV[:, 1]), :]

end


conelmtab (generic function with 1 method)

# Exercise 2.2

In [5]:
function basfun(n,VX,VY,EToV)
    i1 = EToV[n,1]
    i2 = EToV[n,2]
    i3 = EToV[n,3]
    abc = zeros(3,3)

    m = 1
    
    for (j, k) in ((i2, i3), (i3, i1), (i1, i2))
        abc[m, 1] = VX[j]*VY[k] - VX[k]*VY[j] 
        abc[m, 2] = VY[j] - VY[k] 
        abc[m, 3] = VX[k]- VX[j] 
        m += 1
    end


    delta = 1/2 *(abc[1,1] + abc[2,1] + abc[3,1])
    return abc, delta

end

basfun (generic function with 1 method)

In [18]:
function  outernormal(n,k,VX,VY,EToV)
    k2 = ((k+1)%3 +1)
    dx =  VX[EToV[n, k2]] - VX[EToV[n, k]]
    dy =  VY[EToV[n, k2]] - VY[EToV[n, k]]

    norm = [dy; -dx]./sqrt(dx^2+dy^2)

    return norm
end

outernormal (generic function with 1 method)

In [22]:
n = 9
k = 2
x0 = -2.5
y0 = -4.8
L1 = 7.6
L2 = 5.9
noelm1 = 4
noelm2 = 3
VX, VY = xy(x0, y0, L1, L2, noelm1, noelm2)
EToV = conelmtab(noelm1, noelm2)
abc, delta = basfun(n, VX, VY, EToV)
norm = outernormal(n,k,VX,VY,EToV)
display(abc)
display(delta)
display(norm)

3×3 Matrix{Float64}:
  2.55667  -1.96667   0.0
 -1.64667   0.0      -1.9
  2.82667   1.96667   1.9

1.8683333333333336

2-element Vector{Float64}:
 0.7191913900847712
 0.6948120209293551

# Exercise 2.3

In [8]:
function assembly(VX,VY,EToV,lam1,lam2,qt)
    M = length(VX)
    N, _ = size(EToV)

    A = zeros(M,M)
    b = zeros(M)

    for n in 1:N
        abc, delta = basfun(n, VX, VY, EToV)

        for r in 1:3
            i = EToV[n,r]
            b[i] += abs(delta)/9* sum(qt[EToV[n, :]])
            
            for s in 1:3
                j = EToV[n,((s+1)%3 +1)]
                kn = 1/(4*abs(delta))*(lam1*abc[r,2]*abc[s,2] + lam2*abc[r,3]*abc[s,3])
                A[i,j] += kn 
            end
        end
    end
    return A, b

end

assembly (generic function with 1 method)

In [13]:
x0, y0 = (0.0,0.0)
L1 = 1.0
L2 = 1.0
lam1 = 1
lam2 = 1
noelm1 = 4
noelm2 = 4
qt = zeros((noelm1+1)*(noelm2+1))

VX, VY = xy(x0, y0, L1, L2, noelm1, noelm2)
EToV = conelmtab(noelm1, noelm2)

A, b =assembly(VX,VY,EToV,lam1,lam2,qt)

display(A)
display(b)



25×25 Matrix{Float64}:
 -0.5   0.0   0.0   0.0   0.0   0.5  …   0.0   0.0   0.0   0.0   0.0   0.0
  1.0  -1.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   1.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   1.0  -0.5   0.0      0.0   0.0   0.0   0.0   0.0   0.0
 -0.5   0.0   0.0   0.0   0.0  -1.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.5   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -0.5  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  

25-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [ ]:
 
 fd = @(p) -min(min(min(1+p(:,2),1-p(:,2)),1+p(:,1)),1-p(:,1));
  fh = @(p) ones(size(p,1),1);
  [p,t] = distmesh( fd, fh, 0.2, [-1,-1;1,1], [-1,-1;-1,1;1,-1;1,1] );
  patch( 'vertices', p, 'faces', t, 'facecolor', [.9, .9, .9] )

LoadError: LoadError: UndefVarError: `@p` not defined
in expression starting at In[6]:2